In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, NoSuchWindowException, StaleElementReferenceException
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
os.getcwd()
from urllib.request import urlopen as uReq
from time import sleep

In [2]:
#launch url
try:
    driver.close()
except (NameError, NoSuchWindowException):
    pass
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument("--no-startup-window")
driver = webdriver.Chrome(os.getcwd()+'/chromedriver', options = options)
driver.implicitly_wait(8)

In [3]:
def click_on_plus_button(driver):
    buttons = driver.find_elements_by_class_name("location-review-review-list-parts-ExpandableReview__cta--2mR2g")
    for k in range(len(buttons)):
        try:
            buttons_2 = driver.find_elements_by_class_name("location-review-review-list-parts-ExpandableReview__cta--2mR2g")
            buttons_2[k].click()
        except (ElementNotInteractableException, StaleElementReferenceException):
            pass
        
            
        

In [4]:
def click_on_next_button(driver):
    sleep(0.5)
    button = driver.find_element_by_link_text("Next")
    button.click()

In [5]:
def get_grades(review_blocs, reviewer_nb):
    grade_dict = {}
    grades_list = review_blocs[reviewer_nb]\
    .find_element_by_class_name("location-review-review-list-parts-AdditionalRatings__ratings--hIt-r") \
    .find_elements_by_class_name('location-review-review-list-parts-AdditionalRatings__rating--1_G5W')
    for grade in grades_list:
        grade_dict[grade.text] = grade.find_element_by_tag_name('span').find_element_by_tag_name('span').get_attribute("class").split('_')[-1][0]
    return grade_dict

In [6]:
def get_reviews(driver):
    reviews_dict = {
            "member_infos": {},
            "tags": {},
            "general_grade": {},
            "title": {},
            "review_text": {},
            "travel_date": {},
            "grades": {}
        }
    review_blocs = driver.find_elements_by_class_name("location-review-review-list-parts-SingleReview__mainCol--1hApa")
    members_infos = driver.find_elements_by_class_name("social-member-event-MemberEventOnObjectBlock__event_wrap--1YkeG")[:len(review_blocs)]
    for reviewer in range(len(review_blocs)):
        reviews_dict['member_infos'][reviewer] = members_infos[reviewer].text.split('\n')
        reviews_dict['tags'][reviewer] = review_blocs[reviewer].find_element_by_class_name("location-review-review-list-parts-RatingLine__labelsContainer--rSajH").text.split('\n')
        reviews_dict['general_grade'][reviewer] = review_blocs[reviewer].find_element_by_class_name("location-review-review-list-parts-RatingLine__bubbles--GcJvM").find_element_by_tag_name('span').get_attribute("class").split('_')[-1][0]
        reviews_dict['title'][reviewer] = review_blocs[reviewer].find_element_by_class_name("location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT").text
        reviews_dict['review_text'][reviewer] = review_blocs[reviewer].find_element_by_class_name("location-review-review-list-parts-ExpandableReview__reviewText--gOmRC").text.replace('\n',' ')

        try:
            reviews_dict['travel_date'][reviewer] = review_blocs[reviewer].find_element_by_class_name("location-review-review-list-parts-EventDate__event_date--1epHa").text
        except NoSuchElementException:
            travel_date = None
        try:
            reviews_dict['grades'][reviewer] = get_grades(review_blocs, reviewer)
        except NoSuchElementException:
            grades = None
    return reviews_dict

In [7]:
def get_reviews_companies(company_dict):
    company_list = list(company_dict.items())
    for company in company_list:
        try:
            os.mkdir('{}/'.format(company[0]))
        except FileExistsError:
            pass
        
        driver.get(company[1])
        reviews_list = []
        next_page_exist = True
        page = 1
        while next_page_exist:
            click_on_plus_button(driver)
            reviews = pd.DataFrame(get_reviews(driver))
            reviews.to_csv('{}/page_{}.csv'.format(company[0], page), sep = '|')
            reviews_list.append(reviews)
            print('page {} of {} done'.format(page, company[0]))
            try:
                click_on_next_button(driver)
            except:
                next_page_exist = False
                print('{} done'.format(company[0]))
            page += 1
        reviews_df = pd.concat(reviews_list, ignore_index = True)
        reviews_df.to_csv('{}/concat_data_{}.csv'.format(company[0], company[0]), sep = '|', index = False)

In [8]:
company_dict = {
"American Airlines": "https://www.tripadvisor.com/Airline_Review-d8729020-Reviews-American-Airlines",
"Air China": "https://www.tripadvisor.com/Airline_Review-d8729000-Reviews-Air-China",
"Lufthansa": "https://www.tripadvisor.com/Airline_Review-d8729113-Reviews-Lufthansa",
"Emirates": "https://www.tripadvisor.com/Airline_Review-d8729069-Reviews-Emirates",
"All Nippon Airways (ANA)": "https://www.tripadvisor.com/Airline_Review-d8729021-Reviews-ANA-All-Nippon-Airways",
"LATAM Airlines Group": "https://www.tripadvisor.com/Airline_Review-d10290698-Reviews-LATAM-Airlines",
"Air Canada": "https://www.tripadvisor.com/Airline_Review-d8728998-Reviews-Air-Canada",
"Air France": "https://www.tripadvisor.com/Airline_Review-d8729003-Reviews-Air-France",
"Aeroflot Russian Airlines": "https://www.tripadvisor.com/Airline_Review-d8728941-Reviews-Rossiya-Russian-Airlines",
"Singapore Airlines": "https://www.tripadvisor.com/Airline_Review-d8729151-Reviews-Singapore-Airlines",
}

In [ ]:
get_reviews_companies(company_dict)

page 1 of American Airlines done
page 2 of American Airlines done
page 3 of American Airlines done
page 4 of American Airlines done
page 5 of American Airlines done
page 6 of American Airlines done
page 7 of American Airlines done
page 8 of American Airlines done
page 9 of American Airlines done
page 10 of American Airlines done
page 11 of American Airlines done
page 12 of American Airlines done
page 13 of American Airlines done
page 14 of American Airlines done
page 15 of American Airlines done
page 16 of American Airlines done
